In [10]:
import pandas as pd
import numpy as np
import matplotlib as plt
from mlxtend.frequent_patterns import apriori, association_rules
import mysql.connector
import gender_guesser.detector as gender

In [11]:
def solostove_data_import():
    """
    Imports data from bc_staging Solostove database for use in market basket analysis.
    """
    username_entered = input('Enter username:')
    password_entered = input('Enter password:')
    connection = mysql.connector.connect(host='ss-ods-1.cluster-cf8yoeuctpvr.us-east-1.rds.amazonaws.com', database='bc_staging', user= username_entered, password = password_entered)
    data = pd.read_sql("""SELECT vom.order_id, vopm.name, vom.billing_address_state, vom.billing_address_first_name, vom.is_first_order FROM vw_orders_mba vom, vw_order_products_mba vopm 
        WHERE vom.order_id = vopm.order_id ORDER BY vom.order_id DESC LIMIT 500000""", con = connection)
    connection.close()
    return data

In [12]:
def solostove_data_preprocessing(data):
    """
    Data preprocessing for Solostove data
    Inputs:
    data -> Pandas DataFrame with following form
        Cols -> order_id, name
        Rows -> Represent purchase of product, with order_id and the product's name
    Outputs:
    orders_binary -> Pandas DataFrame with following form.
        Cols: column for each product name
        Rows: Represent an order, with True for products in order and False for products not in an order.
    """
    def sale_remove(name):
        return name.rpartition(' (')[0]
    def extra_space_remove(name):
        return name[:-1]
    def encode_units(x):
        if x <= 0:
            return False
        if x >= 1:
            return True
    d = gender.Detector(case_sensitive=False)
    # Removes shipping insurance rows
    data = data[data['name'] != 'Route Shipping Insurance']
    # Removes rows with damage
    data = data[data['name'].str.contains('Dented') == False]
    # Simplifies gift card rows to one product type
    data.loc[data['name'].str.contains('Gift'),'name'] = 'Gift Card'
    # Removes shipping dates
    data.loc[data['name'].str.contains(' \('),'name'] = data.loc[data['name'].str.contains(' \('),'name'].apply(sale_remove)
    # Removes extra spaces
    data.loc[data['name'].str[-1] == ' ', 'name'] = data.loc[data['name'].str[-1] == ' ', 'name'].apply(extra_space_remove)
    
    
    data['bought'] = 1
    orders = data.groupby(['order_id', 'name'])['bought'].sum().unstack().reset_index().fillna(0).set_index('order_id')
    
    orders_binary = orders.applymap(encode_units)
    return orders_binary

In [13]:
def market_basket_analysis(orders_binary):
    """
    Performs market basket analysis on the given dataset, outputing the most common combinations of products bought.
    Inputs: 
    
    data -> Pandas DataFrame with following form.
        Cols: column for each product name
        Rows: Represent an order, with True for products in order and False for products not in an order.
    
    Outputs:
    
    Tuple with two DataFrames, rules and frequent_itemsets
    rules -> Pandas DataFrame with associations between antecedents and consequents, along with statisitics for each pairing
        Antecedents and consequents are the names of products involved in comparison
    frequent_itemsets -> Pandas DataFrame representing the most common combinations of products ordered.
        Includes a set of the item, as well as the probability of that order and the number of items in that order.
    """
    frequent_itemsets = apriori(orders_binary, min_support=0.001, use_colnames=True, max_len = 2)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    
    return (rules, frequent_itemsets)

In [14]:
def solostove_data_postprocessing(outputs):
    """
    Takes association rules and frequent purchase DataFrames, outputing most useful representation of data for Solostove.
    Inputs:
    outputs -> tuple of size 2
        element 1 -> rules
            Pandas DataFrame with associations between antecedents and consequents, along with statisitics for each pairing
        element 2 -> frequent itemsets
            Pandas DataFrame representing the most common combinations of products ordered.
    Outputs:
    recommendations -> Pandas DataFrame representing the products most likely to be paired with each product.
        Includes lift and support metrics
            Lift -> Measures the probability of the two items being ordered together relative to the probabilty of the two items being ordered
            Support -> Measures the probability of the two items being ordered together
    """
    rules = outputs[0]
    frequent_itemsets = outputs[1]
    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x))).astype('unicode')
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x))).astype('unicode')
    
    recommended = rules.groupby(['antecedents', 'consequents'])[['lift', 'support']].first()
    recommended = recommended.sort_values(by = 'support', ascending = False)#level = 'antecedents', sort_remaining = False)
    recommended['set'] = recommended.index
    recommended['set'] = recommended['set'].apply(lambda x: frozenset(x))
    recommended = recommended.drop_duplicates(subset = 'set')
    frequent_itemsets['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: ', '.join(list(x))).astype('unicode')
    frequent_itemsets = frequent_itemsets.set_index('itemsets')
    frequent_itemsets = frequent_itemsets.sort_values(by = 'support', ascending = False)
    return (recommended, frequent_itemsets)

In [15]:
data = solostove_data_import()

Enter username: 
Enter password: 


ProgrammingError: 1045 (28000): Access denied for user 'CORP\jamison'@'54.197.71.94' (using password: NO)

In [ ]:
recommended, frequent_itemsets = solostove_data_postprocessing(market_basket_analysis(solostove_data_preprocessing(data)))

In [ ]:
recommended